In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SaaS Analysis").getOrCreate()


In [2]:
from google.colab import files
uploaded = files.upload()


Saving top_100_saas_companies_2025.csv to top_100_saas_companies_2025.csv


In [5]:
df = spark.read.option("header", True).csv("top_100_saas_companies_2025.csv")
df.printSchema()
df.show()


root
 |-- Company Name: string (nullable = true)
 |-- Founded Year: string (nullable = true)
 |-- HQ: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Total Funding: string (nullable = true)
 |-- ARR: string (nullable = true)
 |-- Valuation: string (nullable = true)
 |-- Employees: string (nullable = true)
 |-- Top Investors: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- G2 Rating: string (nullable = true)

+------------+------------+--------------------+--------------------+-------------+------+-------------------+---------+--------------------+--------------------+---------+
|Company Name|Founded Year|                  HQ|            Industry|Total Funding|   ARR|          Valuation|Employees|       Top Investors|             Product|G2 Rating|
+------------+------------+--------------------+--------------------+-------------+------+-------------------+---------+--------------------+--------------------+---------+
|   Microsoft|        1975

In [7]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import DoubleType

def convert_currency(value):
    if value is None:
        return None
    value = value.replace("$", "").strip()
    try:
        if value.endswith("T"):
            return float(value[:-1]) * 1_000_000_000_000
        elif value.endswith("B"):
            return float(value[:-1]) * 1_000_000_000
        elif value.endswith("M"):
            return float(value[:-1]) * 1_000_000
        else:
            return float(value)
    except:
        return None
currency_udf = udf(convert_currency, DoubleType())

df_converted = df.withColumn("Valuation_Num", currency_udf(col("Valuation"))) \
                 .withColumn("ARR_Num", currency_udf(col("ARR"))) \
                 .withColumn("Funding_Num", currency_udf(col("Total Funding")))

df_converted.select("Company Name", "Valuation", "Valuation_Num", "ARR", "ARR_Num", "Total Funding", "Funding_Num").show(5)


+------------+---------+-------------+------+-------+-------------+-------------------+
|Company Name|Valuation|Valuation_Num|   ARR|ARR_Num|Total Funding|        Funding_Num|
+------------+---------+-------------+------+-------+-------------+-------------------+
|   Microsoft|      $3T|       3.0E12| $270B| 2.7E11|          $1B|              1.0E9|
|  Salesforce|  $227.8B|     2.278E11|$37.9B|3.79E10|       $65.4M|6.540000000000001E7|
|       Adobe|    $240B|       2.4E11|$19.4B|1.94E10|        $2.5M|          2500000.0|
|      Oracle|    $350B|       3.5E11|$52.9B|5.29E10|          $2K|               NULL|
|         SAP|    $215B|      2.15E11|$32.5B|3.25E10|          N/A|               NULL|
+------------+---------+-------------+------+-------+-------------+-------------------+
only showing top 5 rows



In [8]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc

window_spec = Window.partitionBy("Industry").orderBy(desc("Valuation_Num"))

df_ranked = df_converted.withColumn("Rank", rank().over(window_spec))
df_top2 = df_ranked.filter(col("Rank") <= 2)

df_top2.select("Company Name", "Industry", "Valuation", "Valuation_Num", "Rank").show(truncate=False)


+------------+----------------------+--------------+-------------+----+
|Company Name|Industry              |Valuation     |Valuation_Num|Rank|
+------------+----------------------+--------------+-------------+----+
|AppDynamics |APM                   |$3.7B (Cisco) |NULL         |1   |
|Affirm      |BNPL                  |$12B          |1.2E10       |1   |
|Looker      |Business Intelligence |$2.6B (Google)|NULL         |1   |
|CircleCI    |CI/CD                 |$1.7B         |1.7E9        |1   |
|Salesforce  |CRM                   |$227.8B       |2.278E11     |1   |
|Marqeta     |Card Issuing          |$4.3B         |4.3E9        |1   |
|Zscaler     |Cloud Security        |$30B          |3.0E10       |1   |
|Netskope    |Cloud Security        |$7.5B         |7.5E9        |2   |
|Dropbox     |Cloud Storage         |$8.5B         |8.5E9        |1   |
|Box         |Cloud Storage         |$3.5B         |3.5E9        |2   |
|Miro        |Collaboration         |$17.5B        |1.75E10     

In [10]:
# Problem 3: Window Function – ARR Growth Gaps

from pyspark.sql.functions import udf, col, lag, when, split, trim, desc
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType

arr_window = Window.partitionBy("Industry").orderBy(col("ARR_Num").desc())

df_arr_diff = df_converted.withColumn("Prev_ARR", lag("ARR_Num").over(arr_window)) \
                          .withColumn("ARR_Difference", col("Prev_ARR") - col("ARR_Num")) \
                          .filter(col("ARR_Difference") > 1_000_000_000)

df_arr_diff.select("Company Name", "Industry", "ARR_Num", "ARR_Difference").show(truncate=False)


+------------+-------------------+-------+--------------------+
|Company Name|Industry           |ARR_Num|ARR_Difference      |
+------------+-------------------+-------+--------------------+
|Netskope    |Cloud Security     |5.0E8  |1.1E9               |
|Box         |Cloud Storage      |1.0E9  |1.5E9               |
|RingCentral |Communications     |2.2E9  |1.8999999999999995E9|
|CrowdStrike |Cybersecurity      |3.1E9  |4.4E9               |
|Palantir    |Data Analytics     |2.2E9  |1.5E9               |
|Redis       |Database           |1.0E8  |1.6E9               |
|Figma       |Design             |6.0E8  |1.4E9               |
|SAP         |Enterprise Software|3.25E10|2.375E11            |
|Stripe      |Payments           |1.4E10 |5.7E9               |
+------------+-------------------+-------+--------------------+



In [11]:
# Problem 4: CASE WHEN – Label Companies by G2 Rating

df_rating_tier = df_converted.withColumn("Rating_Tier", when(col("G2 Rating") >= 4.7, "Excellent")
                                         .when(col("G2 Rating") >= 4.3, "Very Good")
                                         .when(col("G2 Rating") >= 4.0, "Good")
                                         .otherwise("Average"))

df_rating_tier.select("Company Name", "G2 Rating", "Rating_Tier").show(truncate=False)


+------------+---------+-----------+
|Company Name|G2 Rating|Rating_Tier|
+------------+---------+-----------+
|Microsoft   |4.4      |Very Good  |
|Salesforce  |4.3      |Very Good  |
|Adobe       |4.5      |Very Good  |
|Oracle      |4.0      |Good       |
|SAP         |4.1      |Good       |
|Intuit      |4.4      |Very Good  |
|ServiceNow  |4.4      |Very Good  |
|Workday     |4.2      |Good       |
|Zoom        |4.5      |Very Good  |
|Shopify     |4.4      |Very Good  |
|Atlassian   |4.3      |Very Good  |
|Snowflake   |4.4      |Very Good  |
|HubSpot     |4.4      |Very Good  |
|DocuSign    |4.5      |Very Good  |
|Slack       |4.5      |Very Good  |
|Notion      |4.7      |Excellent  |
|Datadog     |4.4      |Very Good  |
|MongoDB     |4.5      |Very Good  |
|Okta        |4.4      |Very Good  |
|Twilio      |4.3      |Very Good  |
+------------+---------+-----------+
only showing top 20 rows



In [12]:
from pyspark.sql.functions import split, trim

investor_tiers = spark.createDataFrame([
    ("Accel", "Tier 1"),
    ("Sequoia", "Tier 1"),
    ("Andreessen Horowitz", "Tier 1"),
    ("SoftBank", "Tier 2"),
    ("Lightspeed", "Tier 2"),
    ("Unknown", "Tier 3")
], ["Investor", "Tier"])

df_tier = df_converted.withColumn("First_Investor", trim(split(col("Top Investors"), ",")[0])) \
                      .join(investor_tiers, col("First_Investor") == col("Investor"), "left") \
                      .filter(col("Tier").isin("Tier 1", "Tier 2")) \
                      .select("Company Name", "First_Investor", "Tier", "Valuation", "Valuation_Num")

df_tier.show(truncate=False)


+------------------+-------------------+------+--------------------+-------------+
|Company Name      |First_Investor     |Tier  |Valuation           |Valuation_Num|
+------------------+-------------------+------+--------------------+-------------+
|Algolia           |Accel              |Tier 1|$2.3B               |2.3E9        |
|Sumo Logic        |Accel              |Tier 1|$2.3B               |2.3E9        |
|Segment           |Accel              |Tier 1|$3.2B (Twilio)      |NULL         |
|Freshworks        |Accel              |Tier 1|$5.2B               |5.2E9        |
|Slack             |Accel              |Tier 1|$27.7B (Salesforce) |NULL         |
|Netskope          |Sequoia            |Tier 1|$7.5B               |7.5E9        |
|Verkada           |Sequoia            |Tier 1|$3.2B               |3.2E9        |
|Gong              |Sequoia            |Tier 1|$7.3B               |7.3E9        |
|Qualtrics         |Sequoia            |Tier 1|$12.5B (Silver Lake)|NULL         |
|Rin

In [14]:

industry_medians = spark.createDataFrame([
    ("Enterprise Software", 150_000_000_000),
    ("CRM", 100_000_000_000),
    ("AI", 70_000_000_000),
    ("HRTech", 50_000_000_000),
], ["Industry", "Median_Valuation"])

df_median_cmp = df_converted.join(industry_medians, "Industry", "left").withColumn("Valuation_Position", when(col("Valuation_Num") > col("Median_Valuation"), "Above Median").otherwise("Below Median")).select("Company Name", "Industry", "Valuation_Num", "Median_Valuation", "Valuation_Position")

df_median_cmp.show(truncate=False)


+------------+----------------------+-------------+----------------+------------------+
|Company Name|Industry              |Valuation_Num|Median_Valuation|Valuation_Position|
+------------+----------------------+-------------+----------------+------------------+
|Zoom        |Video Communications  |8.5E10       |NULL            |Below Median      |
|Stripe      |Payments              |6.5E10       |NULL            |Below Median      |
|Atlassian   |Collaboration Software|5.5E10       |NULL            |Below Median      |
|Oracle      |Database & Enterprise |3.5E11       |NULL            |Below Median      |
|Workday     |HR & Finance          |6.5E10       |NULL            |Below Median      |
|Slack       |Team Communication    |NULL         |NULL            |Below Median      |
|Adobe       |Creative Software     |2.4E11       |NULL            |Below Median      |
|Microsoft   |Enterprise Software   |3.0E12       |150000000000    |Above Median      |
|SAP         |Enterprise Softwar